In [ ]:
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf 
from tensorflow.keras import models,layers
import keras 

In [ ]:
train_data_path = 'C:/Users/sagos/Desktop/Model/CottonDisease/train'
test_data_path = 'C:/Users/sagos/Desktop/Model/CottonDisease/test'
val_data_path = 'C:/Users/sagos/Desktop/Model/CottonDisease/val'

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
img_size = 224
batch_size = 32
channels = 3

In [ ]:
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [ ]:
training_data = training_datagen.flow_from_directory(train_data_path, 
                                      target_size=(img_size,img_size),
                                      batch_size=batch_size,
                                      class_mode='categorical')

In [ ]:
val_data = training_datagen.flow_from_directory(val_data_path, 
                                      target_size=(img_size,img_size), 
                                      batch_size=batch_size,
                                      class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale =1./255)

In [ ]:
test_data = test_datagen.flow_from_directory(test_data_path,
                                  target_size=(img_size,img_size),
                                  batch_size=batch_size,
                                  class_mode='categorical')

In [ ]:
test_labels = test_data.classes
test_labels

In [ ]:
images = [training_data[0][0][0] for i in range(5)]
plotImages(images)

In [ ]:
inputshape = (img_size ,img_size , 3)

In [ ]:
model = keras.models.Sequential([
    layers.Conv2D(filters = 32 , kernel_size = (3 , 3),activation = 'relu' , input_shape = inputshape),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(filters = 64 , kernel_size = (3,3)  , activation ='relu'),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(filters = 128 , kernel_size = (3,3) , activation = 'relu'),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(filters = 256, kernel_size = (3,3) , activation = 'relu'),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(units = 128 , activation = 'relu'),
    layers.Dropout(0.1),
    layers.Dense(units = 128 , activation = 'relu'),
    layers.Dropout(0.25),
    layers.Dense(units = 4 , activation = 'softmax'),
    
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es = keras.callbacks.EarlyStopping(
        monitor="val_acc",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True, 
     )

rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_acc",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

In [ ]:
history = model.fit(training_data,validation_data= val_data,callbacks=[es, rp],epochs=50,verbose=1)

In [ ]:
class_names={0:'diseased cotton leaf',1:'diseased cotton plant',2:'fresh cotton leaf',3:'fresh cotton plant'}

In [ ]:
test_results=model.evaluate(test_data)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}')
print(f'Test loss: {test_results[0]*100:.3f}%')
print(f'Test accuracy: {test_results[1]*100:.2f}%')

In [ ]:
pred=model.predict(test_data)
y_pred_classes = [np.argmax(element) for element in pred]
y_pred_classes[:20]

In [ ]:
def predict(image_path):
    test_image=image.load_img(image_path, target_size =(img_size,img_size) )
    plt.imshow(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    preds=np.argmax(model.predict(test_image))
    print(class_names[preds])
    

In [ ]:
test_image=image.load_img('C:/Users/sagos/Desktop/Model/CottonDisease/test/diseasedcottonleaf/dis_leaf(1).jpg', target_size = (img_size,img_size))
test_image = np.expand_dims(test_image, axis = 0)
test_image.shape

In [ ]:
#diseased cotton leaf
predict('C:/Users/sagos/Desktop/Model/CottonDisease/test/diseasedcottonleaf/dis_leaf(1).jpg')

In [ ]:
#diseased cotton plant
predict('C:/Users/sagos/Desktop/Model/CottonDisease/test/diseasedcottonplant/diseasedplant(1).jpg')

In [ ]:
#fresh cotton leaf
predict('C:/Users/sagos/Desktop/Model/CottonDisease/test/freshcottonleaf/freshleaf(1).jpg')

In [ ]:
#fresh cotton plant
predict('C:/Users/sagos/Desktop/Model/CottonDisease/test/freshcottonplant/freshplant(1).jpg')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'val_acc'], loc='best')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='best')
plt.show()

In [ ]:
model.save('C:/Users/sagos/Desktop/Model/cottondisease.h5')
model_json = model.to_json()

with open('C:/Users/sagos/Desktop/Model/cottondisease.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('C:/Users/sagos/Desktop/Model/cottondisease.bin')


In [ ]:
print(classification_report(test_labels, y_pred_classes))

In [5]:
import tensorflow as tf
import json

# Load the .h5 model in Keras
model = tf.keras.models.load_model('C:/Users/sagos/Desktop/Model/cottondisease.h5')

# Convert model architecture to JSON
model_json = model.to_json()

# Save model architecture as .json file
with open('C:/Users/sagos/Desktop/Model/cottondisease.json', 'w') as json_file:
    json_file.write(model_json)

# Save model weights as .bin file
model.save_weights('C:/Users/sagos/Desktop/Model/cottondisease.bin')


In [6]:
import json

with open('C:/Users/sagos/Desktop/Model/cottondisease.json', 'r') as json_file:
    data = json.load(json_file)


In [7]:
import pickle

binary_data = pickle.dumps(data)


In [9]:
with open('C:/Users/sagos/Desktop/Model/cottondisease.bin', 'wb') as bin_file:
    bin_file.write(binary_data)
